In [1]:
import os
import swift
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import mpl_toolkits.mplot3d.art3d as art3d
import roboticstoolbox as rtb
import matplotlib
from roboticstoolbox.robot.ERobot import ERobot
from spatialgeometry import Cuboid, Sphere, Cylinder
from spatialmath import SE3
from matplotlib.patches import Rectangle, Circle
from matplotlib.colors import ListedColormap
from collections import defaultdict
from tqdm import tqdm
# matplotlib.use('Qt5Agg')

In [2]:
# Import Panda robot in the environment
robot     = rtb.models.Panda()
robot.q   = robot.qr

print('Initial configuration of robot: ', robot.q)
# robot.plot(robot.q) #, backend = 'swift')

Initial configuration of robot:  [ 0.         -0.3         0.         -2.2         0.          2.
  0.78539816]


In [3]:
# Global Variables
num_obstacles = 8
num_joints    = 7
obs           = [0]*num_obstacles
prob_goal     = 0

# Limits for the robot links
min_q         = [-166, -101, -166, -176, -166, -1, -166]
max_q         = [166, 101, 166, -4, 166, 215, 166]

# Define waypoints
num_waypoints = 9
waypoints     = [0]*num_waypoints
waypoints[0]  = [0.4, 0, 0.4]
waypoints[1]  = [0.63, -0.45, 0.30]
waypoints[2]  = [0.8, 0, 0.2]
waypoints[3]  = [0.63, 0.45, 0.30]
waypoints[4]  = [0, 0.4, 0.2]
waypoints[5]  = [-0.60, 0.40, 0.30]
waypoints[6]  = [-0.8, 0, 0.2]
waypoints[7]  = [-0.60, -0.45, 0.30]
waypoints[8]  = [0, -0.4, 0.2]

# Define seeds for each waypoint
seeds         = [0]*num_waypoints
seeds[0]      = robot.qr.tolist()
seeds[1]      = [-0.673,  1.399, 0.573,  0.159, -0.218,  1.275, -0.514]
seeds[2]      = [0.708,  2.042,  0.019,  0.261,  0.0118,  1.767, 0.013]
seeds[3]      = [0.652, 1.510, 0.108,  0.318,  0.124,  1.328, 0.656]
seeds[4]      = [ 1.005, -0.405,  0.724, -2.856,  0.342,  2.724, 1.684]
seeds[5]      = [ 1.805, 0.585,  0.824, -1.756,  0.342,  2.724, 1.684]
seeds[6]      = [ 2.905, 0.685,  0.224, -1.656,  0.242,  3.024, 1.684]
seeds[7]      = [ 3.751, 0.565,  0.142, -1.656,  0.442,  3.024, 1.684]
seeds[8]      = [-1.005, -0.465, -0.724, -2.856, -0.342,  2.724, -1.713]

# Get robot configurations for waypoints
waypoint_conf = [0]*num_waypoints

for i in range(num_waypoints):
    waypoint_conf[i]  = robot.ikine_LM(SE3(waypoints[i]), seeds[i])[0]

# Convert the waypoint configurations to degrees as per robot links limits
waypoint_conf = (np.asarray(waypoint_conf) * 180 / np.pi)

In [4]:
# Create cuboids for obstacles in the environment
obs[0] = Cuboid (scale =[0.1 , 0.05 , 1.5], pose = SE3(-0.7, -0.3, 0), collision = True,  color = 'red')    
obs[1] = Cuboid (scale =[0.1 , 0.05 , 1.5], pose = SE3(0.7, 0.25, 0), collision = True, color = 'blue')     

In [5]:
# Create cylinders for obstacles in the environment
obs[2] = Cylinder (radius = 0.05, length = 1, pose = SE3(-0.7, 0.25, 0), collision = True, color = 'green')
obs[3] = Cylinder (radius = 0.05, length = 1, pose = SE3(0.7, -0.25, 0), collision = True, color = 'yellow' ) 

In [6]:
# Create spheres for obstacles in the environment
obs[4] = Sphere (radius = 0.1, pose = SE3(0.8, 0, 1), collision = True, color = 'red')
obs[5] = Sphere (radius = 0.1, pose = SE3(-0.8, 0, 1), collision = True, color = 'red')
obs[6] = Sphere (radius = 0.1, pose = SE3(0, 0.4, 1), collision = True, color = 'blue')
obs[7] = Sphere (radius = 0.1, pose = SE3(0, -0.4, 1), collision = True, color = 'blue')

In [7]:
# Setup environment
env = swift.Swift()
env.launch(realtime=True)

# Add robot to the environment
env.add(robot)

# Add obstacles in the environment
for i in range(num_obstacles):
    env.add(obs[i])

In [19]:
import time
robot.q = robot.qr
env.step(0.5)
time.sleep(0.5)

# q = [-0.67307284,  1.39860141, 0.57346981,  0.15855855, -0.21831332,  1.27462099, -0.51436484] # seed for waypoint[1]
# q = [0.708052336,  2.04158321,  0.01959022,  0.26131966,  0.01179127,  1.76735004, 0.01304029] # seed for waypoint[2]
# q = [0.65235819, 1.5103582, 0.1078503,  0.3175529,  0.1244513,  1.32772172, 0.65572857] # seed for waypoint[3]
# q = [ 1.00507843, -0.40501334,  0.72354248, -2.85560976,  0.34245822,  2.72356921, 1.68395639] # seed for waypoint [4]
# q = [ 1.80507843, 0.58501334,  0.82354248, -1.75560976,  0.34245822,  2.72356921, 1.68395639] # seed for waypoint [5]
# q = [ 2.90507843, 0.68501334,  0.22354248, -1.65560976,  0.24245822,  3.02356921, 1.68395639] # seed for waypoint [6]
# q = [ 3.750507843, 0.56501334,  0.142354248, -1.65560976,  0.44245822,  3.02356921, 1.68395639] # seed for waypoint [7]
# q = [-1.00507843, -0.46501334, -0.72354248, -2.85560976, -0.34245822,  2.72356921, -1.71316006] # seed for waypoint [8]

q1 = robot.ikine_LM(SE3(0.63, 0.45, 0.30), seeds[3])[0]
conf1 = np.asarray(robot.fkine(q1))[:3, 3]
print(q1, conf1)
print(collision_free(q1))

robot.q = q1
env.step(0.5)

[0.652 1.51  0.108 0.318 0.124 1.328 0.656] [0.58292589 0.49146196 0.29708692]
False


### Configuration Space of the Robot

In [ ]:
# Initialize configuration space matrix
conf_space = {}
count      = 0

# Loop to check collisions with obstacles and plot configuration space
for q1 in tqdm(range(min_q[0], max_q[0] + 1)):
    for q2 in range(min_q[1], max_q[1] + 1):
        for q3 in range(min_q[2], max_q[2] + 1):
            for q4 in range(min_q[3], max_q[3] + 1):
                for q5 in range(min_q[4], max_q[4] + 1):
                    print(count)
                    for q6 in range(min_q[5], max_q[5] + 1):
                        for q7 in range(min_q[6], max_q[6] + 1):
                            
                            # Convert angles to radians
                            q = np.array([q1, q2, q3, q4, q5, q6, q7])
                            q = q * np.pi / 180

                            # Check collisions with each obstacle
                            if collision_free(q):
                                coord = np.asarray(robot.fkine(q))[:3, 3]
                                conf_space[count] = coord
                                count += 1
                                
np.save("config_space.npy", conf_space)

### Ancillary  Functions for RRT\*, Informed RRT\* and Gaussian Sampling based RRT\*

In [8]:
def collision_free(_q, radians = True):
    
    # Input:
    # _q: joint configuration of the robot
    
    # Output:
    # bool: True if there is no collision, False if there is a collision
    
    if not radians:
         _q = np.asarray(_q) * np.pi / 180
    
    for i in range(num_obstacles):
        if robot.iscollided(_q, obs[i]):
            return False
    
    return True

def SAMPLEFREE(_min_q, _max_q, _q_goal):
    
    # Input:
    # _min_q: Minimum angular limits of all joints (list)
    # _max_q: Maximum angular limits of all joints (list)
    #_q_goal: Goal point robot configuration (numpy array)
    
    # Output:
    # _q: randomly selected configuration from the configuration space (list)
    
    _q     = []
    temp   = [0, 1]
    prob   = [1 - prob_goal, prob_goal]
    sample = np.random.choice(temp, p = prob)
    
    if sample == 1:
        _q = np.copy(_q_goal).tolist()
    
    else: 
        for i in range(num_joints):
            _q.append(np.random.randint(_min_q[i], _max_q[i])) 
    
    return _q

def INFORMEDSAMPLE(_min_q, _max_q, _q_start, _q_goal, _C, _c_max):
    
    # Input:
    # _min_q: Minimum angular limits of all joints (list)
    # _max_q: Maximum angular limits of all joints (list)
    #_q_start: Start point robot configuration (numpy array)
    #_q_goal: Goal point robot configuration (numpy array)
    # _C: Rotation_to_world_frame matrix
    #_c_max: cost of best solution
    
    # Output:
    # _q: randomly selected configuration from the configuration space (list)
    
    _q = []
    
    if _c_max < np.inf:
        _c_min    = np.linalg.norm(_q_goal - _q_start)
        _q_centre = (_q_start + _q_goal) / 2
        _r1       = _c_max / 2
        _ri       = np.sqrt(_c_max**2 - _c_min**2) / 2
        _L        = np.diag([_r1, _ri, _ri, _ri, _ri, _ri, _ri])
        
        _q_ball   = np.zeros(num_joints)
        for i in range(num_joints):
            _q_ball[i] = np.random.uniform()
        
        _q        = (_C @ _L @ _q_ball.reshape(7,1) + _q_centre.reshape(7,1)).reshape(7,).tolist()
    
    else: 
        for i in range(num_joints):
            _q.append(np.random.randint(_min_q[i], _max_q[i])) 
    
    return _q

def GAUSSIANSAMPLE(_min_q, _max_q, _q_mean, _q_goal):
    
    # Input:
    # _min_q: Minimum angular limits of all joints (list)
    # _max_q: Maximum angular limits of all joints (list)
    # _q_mean: Recently added point to the list of vertices (list)
    #_q_goal: Goal point robot configuration (numpy array)
    
    # Output:
    # _q: randomly selected configuration from the configuration space as per Gaussian distribution (list)
    
    temp   = [0, 1]
    prob   = [1 - prob_goal, prob_goal]
    sample = np.random.choice(temp, prob)
    
    if sample == 1:
        _q = np.copy(_q_goal).tolist()
    
    else: 
        _cov          = np.linalg.norm(_q_goal - np.asarray(_q_mean)) * np.eye(num_joints)
        _q            = np.random.multivariate_normal(_q_mean, _cov)

        for i in range(num_joints):
            _q[i] = np.clip(_q[i], _min_q[i], _max_q[i])
    
    return _q.tolist()

def NEAREST(_q_rand, adj_map):
    
    # Input:
    # adj_map: Dictionary of tree vertices and edges
    # _q_rand: randomly selected configuration from the configuration space
    
    # Output:
    # _q_nearest: vertex which is nearest to _q_rand (numpy array)
    
    nodes       = list(adj_map.keys())
    nodes_array = np.array(nodes)
    dist        = np.asarray(_q_rand) - nodes_array
    dist        = np.linalg.norm(dist, axis = 1)
    min_ind     = np.argmin(dist)
    _q_nearest  = nodes[min_ind]
    
    return _q_nearest

def STEER(_q_rand, _q_nearest, _eps):
    
    # Input:
    # _q_rand: randomly selected configuration from the configuration space
    # _q_nearest: vertex which is nearest to _q_rand
    # _eps: constant epsilon
    
    # Output:
    # _q_new: New point which is _eps distance away from _q_nearest (list)
    
    # Check to ensure non-zero norm
    if np.all(np.asarray(_q_rand) == np.asarray(_q_nearest)):
        _q_new = _q_nearest
    
    else:
        q_bar  = np.asarray(_q_rand) - np.asarray(_q_nearest)
        q_bar  = q_bar / np.linalg.norm(q_bar)
        _q_new = np.asarray(_q_nearest) + (_eps * q_bar)
        _q_new = _q_new.round().astype(int)
        
        for i in range(num_joints):
            _q_new[i] = np.clip(_q_new[i], min_q[i], max_q[i])
    
    return _q_new.tolist()

def EDGECOLLISIONFREE(_q_nearest, _q_new):
    
    # Input:
    # _q_nearest: vertex which is nearest to _q_rand
    # _q_new: New point which is _eps distance away from _q_nearest
    
    # Output:
    # bool: Return False if it is a collision in configuration space, otherwise True
    
    _q_nearest = np.asarray(_q_nearest)
    _q_new     = np.asarray(_q_new)
    
    if not collision_free(_q_nearest, radians = False):
        return False
    
    if not collision_free(_q_new, radians = False):
        return False
    
    if (np.linalg.norm(_q_new - _q_nearest) != 0):
        m          = (_q_new - _q_nearest) / np.linalg.norm(_q_new - _q_nearest)

        q_temp     = np.copy(_q_nearest)
        t          = 0
        
        while(np.all(q_temp != _q_new)):
            q_temp = np.round(_q_nearest + (t * m))
            t     += 1
    
            if not collision_free(q_temp, radians = False):
                return False
    
    return True    

def NEAR(_q_new, adj_map, _r):
    
    # Input:
    # adj_map: Dictionary of tree vertices and edges
    # _q_new: New point which is _eps distance away from _q_nearest
    # _r: radius of the circle centered at _q_new
    
    # Output:
    # _q_near: list of nodes which fall inside the circle centered at _q_new (list)
    
    nodes       = list(adj_map.keys())
    nodes_array = np.array(nodes)
    dist        = np.asarray(_q_new) - nodes_array
    dist        = np.linalg.norm(dist, axis = 1)
    neigh_nodes = np.where(dist < _r)
    
    _q_near     = []
    
    for i in neigh_nodes[0]:
        _q_near.append(nodes[i])

    return _q_near

def NEAREST_EE(_q_rand, adj_map):
    
    nodes       = list(adj_map.keys())
    nodes_array = np.array(nodes)
    
    temp_arr2   = np.zeros((nodes_array.shape[0], 3))
    
    for i in range(nodes_array.shape[0]):
        temp_arr2[i, :] = np.asarray(robot.fkine(nodes_array[i, :]))[:3, 3]
        
    ee_pos        = np.asarray(robot.fkine(_q_rand))[:3, 3]
    temp_arr2     = temp_arr2 - np.asarray(ee_pos)
    temp_arr_norm = np.linalg.norm(temp_arr2, axis = 1)
    index         = np.argmin(temp_arr_norm)
    _q_nearest    = nodes[index]
    
    return _q_nearest 

def compute_path(parent_dict,x_goal):
        
    path=[]
    node = x_goal
    while node!=(-1,-1):
#         print(node)
        path.append(node)
        node = parent_dict[node]
    path.reverse()
    return path

### RRT\* and Gaussian Sampling based RRT\* Implementation

In [9]:
def rrt_star(_start_point, _goal_point):
    
    # Initialize V and E
    graph_dict     = defaultdict(list)
    graph_dict[tuple(_start_point)] = []

    # Algorithm variables
    eps            = 7 #steer
    r_star         = 15 #neighbour
    num_iterations = 4000

    # Initialize Cost Matrix
    cost_matrix    = defaultdict(lambda:np.inf)
    cost_matrix[tuple(_start_point)] = 0

    # Initialize Parent dictionary
    parent_dict    = {}
    q_new          = np.copy(_start_point)

    for i in tqdm(range(num_iterations)):
        q_rand    = SAMPLEFREE(min_q, max_q, _goal_point)
#         q_rand    = GAUSSIANSAMPLE(min_q, max_q, q_new, _goal_point)
        q_nearest = NEAREST(q_rand, graph_dict)
        q_new     = STEER(q_rand, q_nearest, eps)
        if EDGECOLLISIONFREE(q_nearest, q_new):
            graph_dict[tuple(q_new)] = []
            q_near = NEAR(q_new,graph_dict, r_star)
            q_min  = q_nearest
            c_min  = cost_matrix[tuple(q_nearest)] + np.linalg.norm(np.asarray(q_nearest) - np.asarray(q_new))

            for _q in q_near:
                if EDGECOLLISIONFREE(_q, q_new):
                    if cost_matrix[tuple(_q)] + np.linalg.norm(np.asarray(_q) - np.asarray(q_new)) < c_min:
                        q_min = list(_q)
                        c_min = cost_matrix[tuple(_q)] + np.linalg.norm(np.asarray(_q) - np.asarray(q_new))                         

            if list(q_new) != q_min:
                parent_dict[tuple(q_new)] = q_min
                cost_matrix[tuple(q_new)] = c_min
                graph_dict[tuple(q_min)].append(q_new)
                

            for _q in q_near:
                if list(_q) != q_min and EDGECOLLISIONFREE(_q, q_new):
                    if cost_matrix[tuple(q_new)] + np.linalg.norm(np.asarray(_q) - np.asarray(q_new)) < cost_matrix[tuple(_q)]:
                        if list(_q) != q_new:
                            parent_dict[tuple(_q)] = q_new
                            cost_matrix[tuple(_q)] = cost_matrix[tuple(q_new)] + np.linalg.norm(np.asarray(_q) - np.asarray(q_new))
                        
        
    # Get optimal path
    print('Goal point in parent dict: ', tuple(_goal_point) in parent_dict)
    print('Goal point in V: ', tuple(_goal_point) in graph_dict)
    
    print('Starting optimal path')
    
    start = tuple(_start_point)
    path1 = np.copy(_goal_point)
    
    if tuple(_goal_point) not in parent_dict:
        c1    = NEAREST(_goal_point, graph_dict)
        if(np.all(c1 == _goal_point)):
            c_temp_neigh = NEAR(list(_goal_point), graph_dict, r_star)
            if list(_goal_point) in c_temp_neigh:
                path1 = np.vstack((path1, np.asarray(c_temp_neigh[0])))
        else:
            path1 = np.vstack((path1, np.asarray(c1)))
            
        
    else: 
        c1    = tuple(_goal_point)
    
    # Optimal path
    while c1 != start:
        if c1 in parent_dict.keys():
            c1    = tuple(parent_dict[c1])
            path1 = np.vstack((path1, np.asarray(c1)))
    
    print('Optimal path found!')
    
    path1 = np.flip(path1, 0)
    
    return graph_dict, path1

### Informed RRT\* Implementation

In [11]:
def informed_rrt_star(_start_point, _goal_point):
    
    # Initialize V, E and X_soln
    graph_dict     = defaultdict(list)
    graph_dict[tuple(_start_point)] = []
    X_soln         = []

    # Algorithm variables
    eps            = 10
    r_star         = 15
    num_iterations = 5000
    c_best         = np.inf
    r_goal         = 0.8 * np.linalg.norm(_goal_point - _start_point) #0.15

    # Initialize Cost Matrix
    cost_matrix    = defaultdict(lambda:np.inf)
    cost_matrix[tuple(_start_point)] = 0

    # Initialize Parent dictionary
    parent_dict    = {}
    
    # Calculate Rotation_to_world_frame matrix
    a1             = (_goal_point - _start_point) / np.linalg.norm(_goal_point - _start_point)
    one_vector     = np.eye(num_joints)[:, 0]
    M              = a1.reshape(7,1) @ one_vector.reshape(1,7)
    U, _, V_T      = np.linalg.svd(M)
    C              = U @ np.diag([1, 1, 1, 1, 1, np.linalg.det(U), np.linalg.det(V_T)]) @ V_T
    
    temp_flag = True
    
    for i in tqdm(range(num_iterations)):
        if len(X_soln) != 0:
            cost  = np.inf
            for _x in X_soln:
                temp_cost = cost_matrix[tuple(_x)] + np.linalg.norm(_goal_point - np.asarray(_x))
                if temp_cost < cost:
                    cost = temp_cost
            c_best = cost
            if temp_flag:
                print('cbest: ', c_best)
                temp_flag = False
            
        q_rand    = INFORMEDSAMPLE(min_q, max_q, _start_point, _goal_point, C, c_best)
        q_nearest = NEAREST(q_rand, graph_dict)
        q_new     = STEER(q_rand, q_nearest, eps)
        
        if EDGECOLLISIONFREE(q_nearest, q_new):
            graph_dict[tuple(q_new)] = []
            q_near = NEAR(q_new,graph_dict, r_star)
            q_min  = q_nearest
            c_min  = cost_matrix[tuple(q_nearest)] + np.linalg.norm(np.asarray(q_nearest) - np.asarray(q_new))
            
            for _q in q_near:
                if EDGECOLLISIONFREE(_q, q_new):
                    if cost_matrix[tuple(_q)] + np.linalg.norm(np.asarray(_q) - np.asarray(q_new)) < c_min:
                        q_min = list(_q)
                        c_min = cost_matrix[tuple(_q)] + np.linalg.norm(np.asarray(_q) - np.asarray(q_new))                         
 
            if list(q_new) != q_min:
                parent_dict[tuple(q_new)] = q_min
                cost_matrix[tuple(q_new)] = c_min
                graph_dict[tuple(q_min)].append(q_new)

            for _q in q_near:
                if list(_q) != q_min and EDGECOLLISIONFREE(_q, q_new):
                    if cost_matrix[tuple(q_new)] + np.linalg.norm(np.asarray(_q) - np.asarray(q_new)) < cost_matrix[tuple(_q)]:
                        if list(_q) != q_new:
                            parent_dict[tuple(_q)] = q_new
                            cost_matrix[tuple(_q)] = cost_matrix[tuple(q_new)] + np.linalg.norm(np.asarray(_q) - np.asarray(q_new))
            
            if (np.linalg.norm(np.asarray(q_new) - _goal_point) < r_goal):
                X_soln.append(q_new)
        
    # Get optimal path
    
    print('Starting optimal path')
    
    start = tuple(_start_point)
    path1 = np.copy(_goal_point)
    
    if tuple(_goal_point) not in parent_dict:
        c1    = NEAREST(_goal_point,graph_dict)
        if(np.all(c1 == _goal_point)):
            c_temp_neigh = NEAR(list(_goal_point),graph_dict, r_star)
            if list(_goal_point) in c_temp_neigh:
                path1 = np.vstack((path1, np.asarray(c_temp_neigh[0])))
        else:
            path1 = np.vstack((path1, np.asarray(c1)))
            
        
    else: 
        c1    = tuple(_goal_point)
    
    # Optimal path
    while c1 != start:
        if c1 in parent_dict.keys():
            c1    = tuple(parent_dict[c1])
            path1 = np.vstack((path1, np.asarray(c1)))
    
    print('Optimal path found!')
    
    path1 = np.flip(path1, 0)
    
    return graph_dict, parent_dict, path1

In [ ]:
# Call rrt_star to get optimal path
# path1: optimal path

start_pt      = waypoint_conf[0]
goal_pt       = waypoint_conf[1]

print("Start point: {} and goal point: {}".format(start_pt, goal_pt))

graph_dict, path1   = rrt_star(start_pt, goal_pt)
path1               = path1 * np.pi / 180
np.save("path1.npy", path1)
print(path1)

In [ ]:
# Call informed_rrt_star to get optimal path
# path1: optimal path

start_pt      = waypoint_conf[0]
goal_pt       = waypoint_conf[7]

print("Start point: {} and goal point: {}".format(start_pt, goal_pt))

graph_dict, parent_dict, path1   = informed_rrt_star(start_pt, goal_pt)
path1                            = path1 * np.pi / 180
np.save("path1.npy", path1)
print(path1)

In [13]:
# Animate the path in swift

import time
time.sleep(2)

for _q in path1:
    robot.q = _q.tolist()
    env.step(0.5)

time.sleep(3)

robot.q = robot.qr
env.step(0.5)

In [11]:
def add_rectangle (xy , width , height , ax, _color):
    '''
    Add a rectangle patch to the environment
    : param xy: xy corner of the left - bottom corner
    : param height : height of the window
    : param width : width of the window
    : param ax: the axis on which to plot
    : _color: color for the patch
    '''
    obj_patch = Rectangle (xy , width , height, color = _color)
    ax.add_patch ( obj_patch )

def add_circle (xy , radius , ax, _color):
    '''
    Add a circular patch to the environment
    : param xy: center of the circle
    : param radius : radius of the circle
    : param ax: the axis on which to plot
    : _color: color for the patch
    '''
    obj_patch = Circle (xy , radius, color = _color)
    ax.add_patch ( obj_patch )

In [37]:
def plot_tree_path(_conf_space, _graph_dict, _start, _goal, _path1):
    
    # Create a figure and an axis object
    fig, ax = plt.subplots()
    
    # Add obstacles in the environment
    add_rectangle([0.6, 0.225], 0.05, 0.1, ax, 'b')   # obs[0]
    add_rectangle([-0.8, -0.275], 0.05, 0.1, ax, 'r') # obs[1]
    add_circle([-0.7, 0.25], 0.05, ax, 'g')           # obs[2]
    add_circle([0.7, -0.2], 0.05, ax, 'yellow')       # obs[3]
    add_circle([0.8, 0], 0.1, ax, 'r')                # obs[4]
    add_circle([-0.8, 0], 0.1, ax, 'r')               # obs[5]
    add_circle([0, 0.4], 0.1, ax, 'b')                # obs[6]
    add_circle([0, -0.4], 0.1, ax, 'b')               # obs[7]
    
    # Initialize an empty line object
    line, = ax.plot([], [])
    
    # Plot vertices of the tree
    for _q in _graph_dict.keys():
        _q      = np.asarray(_q) * np.pi / 180
        _ee_pos = np.asarray(robot.fkine(_q))[:3, 3]
        plt.scatter(_ee_pos[0], _ee_pos[1], color = 'r', s = 4)

    # Plot edges of the tree
    for _q1 in _graph_dict.keys():
        for _q2 in _graph_dict[_q1]:
            
            _q1     = np.asarray(_q1) * np.pi / 180
            _q2     = np.asarray(_q2) * np.pi / 180
        
            _ee_pos1 = np.asarray(robot.fkine(_q1))[:3, 3]
            _ee_pos2 = np.asarray(robot.fkine(_q2))[:3, 3]
        
            plt.plot([_ee_pos1[0], _ee_pos2[0]], [_ee_pos1[1], _ee_pos2[1]], linewidth = 1, color = 'lightskyblue')
    
     # Plot optimal path
    _ee_pos = np.zeros((_path1.shape[0], 3))
    
    for i in range(_path1.shape[0]):
        _ee_pos[i, :] = np.asarray(robot.fkine(_path1[i, :]))[:3, 3]
    
#     plt.plot(_ee_pos[:, 0], _ee_pos[:, 1], color = 'deeppink', linewidth = 3)
    
    # Show start and goal points
    _start      = np.asarray(robot.fkine(_start * np.pi / 180))[:3, 3]
    _goal       = np.asarray(robot.fkine(_goal * np.pi / 180))[:3, 3]
    plt.scatter(_start[0], _start[1], color = 'g', s = 64)
    plt.scatter(_goal[0], _goal[1], color = 'r', s = 64)
    plt.text(_start[0] - 0.05, _start[1] - 0.05, 'start', fontsize=8, color = 'brown')
    plt.text(_goal[0] - 0.05, _goal[1] - 0.05, 'goal', fontsize=8, color = 'green')
    
    # Animation update function
    def update(frame):
        # Select points up to the current frame
        current_points = _ee_pos[:frame+1]
        print(frame)
        # Update the line data
        line.set_data(current_points[:, 0], current_points[:, 1])
        
        line.set_color('deeppink')
        line.set_linewidth(4)

        return line,

    # Create the animation
    animation = matplotlib.animation.FuncAnimation(fig, update, frames=len(_ee_pos), interval=100, blit=True)

    # Save the animation as a GIF
    animation.save('c_space1.gif', writer='pillow')
    
    # Show the plot
    plt.xlabel('X-coord of End Effector')
    plt.ylabel('Y-coord of End Effector')
    plt.title('End effector trajectory using Informed RRT*')
    plt.savefig('img_08.jpg')
    plt.show()

In [ ]:
# Call the plotting function to plot paths and obtain GIF in configuration space
path1      = np.load("path08.npy")
dict1      = np.load("graph08.npy", allow_pickle = True)
start_pt   = waypoint_conf[0]
goal_pt    = waypoint_conf[8]
graph_dict = dict1.item()
plot_tree_path([], graph_dict.copy(), start_pt, goal_pt, path1)